In [1]:
import sys
sys.path.append("../")

from fgi import *
from torch import randn
from uuid import uuid4

In [2]:
PHI_DIM = 128

In [3]:
# Biểu diễn bài toán
class FunnyRepresent(SoftRepresentUnit):
    def __init__(self, _id ,*args, **kwargs):
        super().__init__(_id, *args, **kwargs)
    
    def forward(self, x, *args, **kwargs):
        return x

In [4]:
class FunnyOP:
    def __init__(self, digit, cute, *args, **kwargs):
        self.digit = digit
        self.cuteness = cute
    
    def __str__(self):
        return f"{self.digit}, {self.cuteness}"
    
    def call_me(self):
        if self.digit == "1":
            print(f"Tôi là số {self.digit}!")
    
    def am_cute(self):
        return "YES" if self.cuteness else "NO"

In [7]:
# Bản dựng cho giải quyết vấn đề
class FunnyProblem(NonCodeProblem):
    def __init__(self, _id, phi_dim ,*args, **kwargs):
        super().__init__(_id, *args, **kwargs)

        self._represent = RepresentLayer.from_units([
            FunnyRepresent(_id=str(uuid4()))
            for _ in range(4)
        ], _id)
        self._co_represent = CoRepresentLayer.from_units([
            CoRepresentUnit(str(uuid4()), 4, phi_dim)
            for _ in range(2)
        ], _id)
        self._property = PropertyLayer.from_units([
            PropertyUnit(str(uuid4()), phi_dim)
            for _ in range(3)
        ], _id)
        self._co_property : CoPropertyLayer = CoPropertyLayer.from_units([
            ChooseOptions(str(uuid4()), 3, options=["1", "2", "3"], property_name="digit", phi_dim=phi_dim),
            Boolean(str(uuid4()), 3, phi_dim, property_name="cute", threshold=0.6)
        ], _id)

        self._update_additional_infor()
        
    def forward(self, x, *args, **kwargs):
        x = self._represent(x)
        x = self._co_represent(x)
        x = self._property(x)
        x = self._co_property(x)
        return x
    
    def recognize_unknown(self, *args, **kwargs):
        pass

    @property
    def _as_object(self, *args, **kwargs):
        return FunnyOP
    
    def as_instance(self, x, skip_inference : bool = False, *args, **kwargs):
        if not skip_inference:
            x = self.forward(x)
        data = self._co_property.intepret(x)
        data.update(**kwargs)
        return self._as_object(**data)

In [8]:
helloworld = FunnyProblem("haha", PHI_DIM)

In [7]:
# In thử kiến trúc mô hình
print(helloworld)

FunnyProblem(
  (_represent): RepresentLayer(
    (_units): ModuleList(
      (0-3): 4 x FunnyRepresent()
    )
  )
  (_co_represent): CoRepresentLayer(
    (_units): ModuleList(
      (0-1): 2 x CoRepresentUnit(
        (_memory): SoftMemoryUnit()
        (_combine): Linear(in_features=256, out_features=128, bias=True)
        (_activate): SiLU()
      )
    )
  )
  (_property): PropertyLayer(
    (_units): ModuleList(
      (0-2): 3 x PropertyUnit(
        (_memory): SoftMemoryUnit()
        (_evaluation): Linear(in_features=128, out_features=1, bias=True)
        (_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (_co_property): CoPropertyLayer(
    (_units): ModuleList(
      (0): ChooseOptions(
        (_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (_enhance): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ELU(alpha=1.0)
        )
        (_decides): Linear(in_features=128, out_

In [9]:
# Thử dữ liệu ngẫu nhiên
x = randn(1, PHI_DIM)
#print(x)
y = helloworld(x)
print(y)

{'digit': tensor([[0.3357, 0.3304, 0.3340]], grad_fn=<SoftmaxBackward0>), 'cute': tensor([[0.5249]], grad_fn=<SigmoidBackward0>)}


In [ ]:
entity = helloworld.as_instance(y, skip_inference=True)

In [11]:
print(entity)

1, False


In [13]:
entity.am_cute()

'NO'

In [12]:
# Thử in các tham số
for name, param in helloworld.state_dict().items():
    print(f"Name: {name}, Size: {param.size()}")

Name: _co_represent._units.0._weighted, Size: torch.Size([4])
Name: _co_represent._units.0._memory._beta, Size: torch.Size([])
Name: _co_represent._units.0._memory._patterns, Size: torch.Size([128, 128])
Name: _co_represent._units.0._combine.weight, Size: torch.Size([128, 256])
Name: _co_represent._units.0._combine.bias, Size: torch.Size([128])
Name: _co_represent._units.1._weighted, Size: torch.Size([4])
Name: _co_represent._units.1._memory._beta, Size: torch.Size([])
Name: _co_represent._units.1._memory._patterns, Size: torch.Size([128, 128])
Name: _co_represent._units.1._combine.weight, Size: torch.Size([128, 256])
Name: _co_represent._units.1._combine.bias, Size: torch.Size([128])
Name: _property._units.0._memory._beta, Size: torch.Size([])
Name: _property._units.0._memory._patterns, Size: torch.Size([128, 128])
Name: _property._units.0._evaluation.weight, Size: torch.Size([1, 128])
Name: _property._units.0._evaluation.bias, Size: torch.Size([1])
Name: _property._units.0._norm.weig

In [13]:
# Tính tổng số lượng tham số
print(sum(p.numel() for p in helloworld.parameters()))

183706


In [14]:
# Thử nghiệm một số phương thức
# Phương thức lấy ra các Layer
# members = inspect.getmembers(helloworld)
# print(members)

In [15]:
# def get_layer(member_value):
#     return issubclass(type(member_value), StaticLayer)

# filter_members = inspect.getmembers(helloworld, get_layer)

In [16]:
# for name, value in filter_members:
#     print(name)

In [17]:
helloworld.layer_names

['_co_property', '_co_represent', '_property', '_represent']

In [18]:
helloworld.units

['6f84d689-f8f6-436c-b1c8-fc2000464087',
 'eb88555d-cdd7-4b77-a989-4f829556804d',
 'ab6d3933-5803-41d6-ba6c-8e8decdecae7',
 '2bac83c1-3ec5-4a6b-a8a0-8e33f682c547',
 '8198fe0e-dc47-49b8-ae06-961a3f0ea89b',
 '01645218-d6bc-4159-afe5-0f617342f7b8',
 '5c0fa921-043b-4082-b142-d46f410c0d20',
 'b07a3a67-02cd-4c31-bfb8-17815b8af156',
 'fde6e05c-78e4-4cef-9d3d-05a3a2f10159',
 'a7e3abd8-1335-4913-8d9f-5172b8b1b155',
 '806d7fa8-49ac-4a8b-a175-66e33dbd2c74']

In [19]:
helloworld.metadata

{'type': 'FunnyProblem',
 'detail': {'_co_property': {'type': 'CoPropertyLayer',
   'components': {'6f84d689-f8f6-436c-b1c8-fc2000464087': {'type': 'ChooseOptions',
     'property': 'digit',
     'options': ['1', '2', '3']},
    'eb88555d-cdd7-4b77-a989-4f829556804d': {'type': 'Boolean',
     'property': 'cute',
     'threshold': 0.6}}},
  '_co_represent': {'type': 'CoRepresentLayer',
   'components': {'ab6d3933-5803-41d6-ba6c-8e8decdecae7': {'type': 'CoRepresentUnit'},
    '2bac83c1-3ec5-4a6b-a8a0-8e33f682c547': {'type': 'CoRepresentUnit'}}},
  '_property': {'type': 'PropertyLayer',
   'components': {'8198fe0e-dc47-49b8-ae06-961a3f0ea89b': {'type': 'PropertyUnit'},
    '01645218-d6bc-4159-afe5-0f617342f7b8': {'type': 'PropertyUnit'},
    '5c0fa921-043b-4082-b142-d46f410c0d20': {'type': 'PropertyUnit'}}},
  '_represent': {'type': 'RepresentLayer',
   'components': {'b07a3a67-02cd-4c31-bfb8-17815b8af156': {'type': 'FunnyRepresent'},
    'fde6e05c-78e4-4cef-9d3d-05a3a2f10159': {'type': '

In [20]:
helloworld.save()

NotImplementedError: Phương thức save chưa được định nghĩa!